In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
%matplotlib inline

In [2]:
def age_10split(x):
    t = int(x['Age'])
    if (t>=0) and (t<10):
        return 0
    elif (t>=10) and (t<20):
        return 1
    elif (t>=20) and (t<30):
        return 2
    elif (t>=30) and (t<40):
        return 3
    elif (t>=40) and (t<50):
        return 4
    elif (t>=50) and (t<60):
        return 5
    elif (t>=60) and (t<70):
        return 6
    elif (t>=70):
        return 7
    
def name_split(x):
    ret = 'Unknown'
    t = str(x['Name'])
    sex = str(x['Sex'])
    t = t.split()
    for i in t:
        if(sex == 'male'):
            if (i == 'Master.'):
                return i
            else:
                ret = 'Mr.'
        elif(sex == 'female'):
            if(i == 'Mrs.'):
                return i
            else:
                ret = 'Miss.'
        else:
            ret = 'Unknown'
    return ret

def Cabin_split(x):
    t = x['Cabin']
    c = str(t[0])
    if c == 'A' or c == 'a':
        return 0
    elif c == 'B' or c == 'b':
        return 1
    elif c == 'C' or c == 'c':
        return 2 
    elif c == 'D' or c == 'd':
        return 3 
    elif c == 'E' or c == 'e':
        return 4 
    elif c == 'F' or c == 'f':
        return 5
    elif c == 'G' or c == 'g':
        return 6
    elif c == 'H' or c == 'h':
        return 7
    elif c == 'I' or c == 'i':
        return 8
    else:
        return 9
    
def ticket_split(x,DF):
    s = str(x['Ticket'])
    n = (len(DF.Ticket.loc[DF.Ticket == s].index.tolist()))
    if n == 1:
        return 1
    elif n == 2:
        return 2
    elif n == 3:
        return 3
    elif n == 4:
        return 4
    elif n == 5:
        return 5
    elif n == 6:
        return 6
    elif n > 6:
        return 7

def Fare_rounddown(x):
    s = int(x['Fare'])
    return s

def Fare_split(x):
    s = int(x['Fare_Rounddown'])
    if (s >= 0) and (s < 10):
        return 0
    elif (s >= 10) and (s < 20):
        return 1
    elif (s >= 20) and (s < 30):
        return 2
    elif (s >= 30) and (s < 40):
        return 3
    elif (s >= 40) and (s < 50):
        return 4
    elif (s >= 50) and (s < 60):
        return 5
    elif (s >= 60) and (s < 70):
        return 6
    elif (s >= 70) and (s < 80):
        return 7
    elif (s >= 80) and (s < 90):
        return 8
    elif (s >= 90) and (s < 100):
        return 9
    elif (s >= 100) and (s < 110):
        return 10
    elif (s >= 110) and (s < 120):
        return 11
    elif (s >= 120) and (s < 130):
        return 12
    elif (s >= 130):
        return 13
    

def concat_SibSp_Parch(x):
    t1 = int(x['SibSp'])
    t2 = int(x['Parch'])
    if (t1+t2+1 > 7):
        return 7
    return t1+t2+1
    
def is_Alone(x):
    t = int(x['SibParch'])
    if t == 1:
        return 1
    return 0

def has_Cabin(x):
    t = str(x['Cabin'])
    if t == 'Unknown':
        return 0
    else:
        return 1

titanic_feature_observationにて新たに作成したcsvファイルを読み込む

In [11]:
# CSVファイルを読み込む
train_df = pd.read_csv('new_titanic_train.csv')
test_df = pd.read_csv('new_titanic_test.csv')

# Unnamed が邪魔なので削除する
drop_columns = ['Unnamed: 0']
train_df = train_df.drop(drop_columns, axis=1)
test_df = test_df.drop(drop_columns, axis=1)

ここではAgeのNaNを予測するモデルをtrain.csvから作成して、予測値の年齢でSurvivedの識別モデルを構築する      
教師データとしてAgeが必要なので欠損値は取り除く

In [12]:
train_df2 = train_df.dropna(subset=['Age']).copy()

In [14]:
#訓練に使わない属性を除く
train2_drop_columns = ['PassengerId','Survived','Name','title_name','Sex','SibSp','Parch','Ticket','Fare','Fare_Rounddown','Cabin','isCabin','Embarked','onehot_Embarked']
train_df2 = train_df2.drop(train2_drop_columns, axis=1)

In [15]:
# データを訓練データ、テストデータに分割する
from sklearn.model_selection import train_test_split

train2, test2 = train_test_split(train_df2, test_size = 0.2)
# indexをリセットする
train2 = train2.reset_index(drop=True)
test2 = test2.reset_index(drop=True)
print(len(train2),len(test2))

571 143


In [16]:
train2.head(2)

,Pclass,onehot_title,onehot_sex,Age,SibParch,Ticket_num,Fare_Rounddown_Split
0,3,0,1,2.0,6,6,3
1,3,2,1,45.0,1,1,0


In [17]:
# 特徴として使うデータとラベルとして使うデータに分ける
X_train = np.array(train2.drop('Age', axis=1).values).astype(np.float32)
y_train = np.array(train2['Age'].values).astype(np.float32)

X_test = np.array(test2.drop('Age', axis=1).values).astype(np.float32)
y_test = np.array(test2['Age'].values).astype(np.float32)

print(X_train.shape,y_train.shape)

(571, 6) (571,)


年齢の予測、回帰をXGboostで行う

In [19]:
from xgboost import XGBRegressor
xg_clf2 = XGBRegressor(max_delta_step=0,gamma=0.05)

xg_model2 = xg_clf2.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [20]:
from sklearn.metrics import log_loss
xg_pred = xg_clf2.predict(X_test)
xg_pred = xg_pred.astype(np.int32)
print('XGBoost mean square loss = {}'.format(np.mean(np.square(y_test - xg_pred))))

XGBoost mean square loss = 113.04672307719618


Ageが欠損しているインデックスリストを作成する

In [21]:
idx = []
count = 0
for v in train_df['Age'].isnull():
    if v:
        idx.append(count)
    count += 1

Ageの欠損を予測モデルの値にて置換する

In [22]:
for i in idx:
    predict = xg_clf2.predict(np.array([train_df['Pclass'].iloc[1],train_df['onehot_title'].iloc[1],
          train_df['onehot_sex'].iloc[1],train_df['SibParch'].iloc[1],
          train_df['Ticket_num'].iloc[1],train_df['Fare_Rounddown_Split'].iloc[1]]).reshape(-1,6))
    train_df['Age'].iloc[i] = predict.astype(np.int32)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Ageの欠損値がないか確認する

In [23]:
train_df.Age.isnull().sum()

0

テストデータに対しても同様の処理を行う

In [24]:
idx = []
count = 0
for v in test_df['Age'].isnull():
    if v:
        idx.append(count)
    count += 1

In [25]:
for i in idx:
    predict = xg_clf2.predict(np.array([test_df['Pclass'].iloc[1],test_df['onehot_title'].iloc[1],
          test_df['onehot_sex'].iloc[1],test_df['SibParch'].iloc[1],
          test_df['Ticket_num'].iloc[1],test_df['Fare_Rounddown_Split'].iloc[1]]).reshape(-1,6))
    test_df['Age'].iloc[i] = predict.astype(np.int32)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [26]:
test_df.Age.isnull().sum()

0

穴埋めしたAgeから10歳ごとに年齢分けしたBy_Age_Classという属性を新たに作成する

In [27]:
train_df['By_Age_Class'] = train_df.apply(lambda x: age_10split(x), axis=1)
test_df['By_Age_Class'] = test_df.apply(lambda x: age_10split(x), axis=1)

In [28]:
train_df.isnull().sum()

PassengerId             0
Survived                0
Pclass                  0
Name                    0
title_name              0
onehot_title            0
Sex                     0
onehot_sex              0
Age                     0
SibSp                   0
Parch                   0
SibParch                0
Ticket                  0
Ticket_num              0
Fare                    0
Fare_Rounddown          0
Fare_Rounddown_Split    0
Cabin                   0
isCabin                 0
Embarked                0
onehot_Embarked         0
By_Age_Class            0
dtype: int64

In [29]:
test_df.isnull().sum()

PassengerId             0
Pclass                  0
Name                    0
title_name              0
onehot_title            0
Sex                     0
onehot_sex              0
Age                     0
SibSp                   0
Parch                   0
SibParch                0
Ticket                  0
Ticket_num              0
Fare                    0
Fare_Rounddown          0
Fare_Rounddown_Split    0
Cabin                   0
isCabin                 0
Embarked                0
onehot_Embarked         0
By_Age_Class            0
dtype: int64

生存者の予測に必要なデータの穴埋めが完了したので生存者の予測モデルを作成する      
予測に必要のないカラムを消去する

In [30]:
train_drop_columns = ['PassengerId','Name','title_name','Sex','Age','SibSp','Parch','Ticket','Ticket_num','Fare','Fare_Rounddown','Cabin','Embarked','onehot_Embarked']
train_df_pred = train_df.drop(train_drop_columns, axis=1)

In [31]:
train_df_pred.head(2)

,Survived,Pclass,onehot_title,onehot_sex,SibParch,Fare_Rounddown_Split,isCabin,By_Age_Class
0,0,3,2,1,2,0,0,2
1,1,1,3,0,2,7,1,3


In [32]:
# データを訓練データ、テストデータに分割する
from sklearn.model_selection import train_test_split

train, test = train_test_split(train_df_pred, test_size = 0.4)
# indexをリセットする
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
print(len(train),len(test))

534 357


In [33]:
# 特徴として使うデータとラベルとして使うデータに分ける
X_train = np.array(train.drop(['Survived'], axis=1).values).astype(np.float32)
y_train = np.array(train['Survived'].values).astype(np.float32)

X_test = np.array(test.drop(['Survived'], axis=1).values).astype(np.float32)
y_test = np.array(test['Survived'].values).astype(np.float32)

print(X_train.shape,y_train.shape)

(534, 7) (534,)


ランダムフォレストとXGboostにてモデルの精度を比較する

In [36]:
from sklearn.ensemble import RandomForestRegressor

rf_clf = RandomForestRegressor()
rf_model = rf_clf.fit(X_train,y_train)

In [37]:
from xgboost import XGBRegressor
xg_clf = XGBRegressor(max_delta_step=10,gamma=0.05)
# モデルを訓練する
xg_model = xg_clf.fit(X_train, y_train)

In [38]:
from sklearn.metrics import log_loss
rf_pred = rf_clf.predict(X_test)
xg_pred = xg_clf.predict(X_test)
rf_pred = np.fmin(0.999,rf_pred)
rf_pred = np.fmax(0.0001,rf_pred)
xg_pred = np.fmin(0.999,xg_pred)
xg_pred = np.fmax(0.0001,xg_pred)
print('RF LogisticRegression log loss = {}'.format(log_loss(y_test,rf_pred)))
print('XGBoost log loss = {}'.format(log_loss(y_test,xg_pred)))

RF LogisticRegression log loss = 0.6894748990015678
XGBoost log loss = 0.41576645462061224


生存者の予測モデルをテストデータに対して適用する

In [39]:
test_drop_columns = ['PassengerId','Name','title_name','Sex','Age','SibSp','Parch','Ticket','Ticket_num','Fare','Fare_Rounddown','Cabin','Embarked','onehot_Embarked']
test_df_pred = test_df.drop(test_drop_columns, axis=1)

In [40]:
test_df_pred.head(2)

,Pclass,onehot_title,onehot_sex,SibParch,Fare_Rounddown_Split,isCabin,By_Age_Class
0,3,2,1,1,0,0,3
1,3,3,0,2,0,0,4


In [41]:
X_pred = np.array(test_df_pred.values.astype(np.float32))

In [42]:
test_pred = xg_clf.predict(X_pred)

In [43]:
# 提出ファイルの作成
submit_df = pd.read_csv('gender_submission.csv')
for i in range(0,len(submit_df)):
    if test_pred[i] < 0.5:
        submit_df['Survived'].iloc[i] = 0
    else:
        submit_df['Survived'].iloc[i] = 1

In [44]:
# 提出ファイルを別の名前で保存する
submit_df.to_csv('titanic_submission.csv',index=False)

In [45]:
show = pd.read_csv('titanic_submission.csv')
show.head(10)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0
